In [1]:
# AWS Public IP Ranges
#https://ip-ranges.amazonaws.com/ip-ranges.json
import requests
import json
import sys
!{sys.executable} -m pip install PyAthena
from pyathena import connect
!{sys.executable} -m pip install netaddr

In [2]:
# Amazon URLs
url = 'https://ip-ranges.amazonaws.com/ip-ranges.json'
resp = requests.get(url=url)
data = resp.json() # Check the JSON Response Content documentation below

In [3]:
#from netaddr import *

In [4]:
#print (json.dumps(data, indent=1))

In [5]:
cloud_ip_addrs = {}

# Row = cidr, ipType, vendor, region, service

In [6]:
c=0
cidr = ''
ipType = 6
vendor = 'AWS'
region = ''
service = ''
for p in data['ipv6_prefixes']:
    #print (p)
    cidr    = p['ipv6_prefix']
    region  = p['region']
    service = p['service']
    
    if cidr in cloud_ip_addrs:
#        print ('duplicate cidr', cidr, p, '\nMatches:', cloud_ip_addrs[cidr])
        cloud_ip_addrs[cidr]['addtl'].append( region+':'+service )
    else:
        cloud_ip_addrs[cidr] = {
            "ipType" : ipType,
            "vendor" : vendor,
            "region" : region,
            "service" :service,
            'addtl' : [ region+':'+service ]
        }
#print (json.dumps(cloud_ip_addrs, indent=1))

In [7]:
c=0
cidr = ''
ipType = 4
vendor = 'AWS'
region = ''
service = ''

for p in data['prefixes']:
    #print (p)
    cidr    = p['ip_prefix']
    region  = p['region']
    service = p['service']
    
    if cidr in cloud_ip_addrs:
#        print ('duplicate cidr', cidr, p, '\nMatches:', cloud_ip_addrs[cidr])
        cloud_ip_addrs[cidr]['addtl'].append( region+':'+service )
    else:
        cloud_ip_addrs[cidr] = {
            "ipType" : ipType,
            "vendor" : vendor,
            "region" : region,
            "service" :service,
            'addtl' : [ region+':'+service ]
        }
#print (json.dumps(cloud_ip_addrs, indent=1))

#print (json.dumps(data, indent=1))
# Services share IP, all duplicates
service = {}
AWS_ips = {}
c=0
for p in data['prefixes']:
    print (p)
    myip = IPNetwork(p['ip_prefix'])
    for i in IPNetwork(p['ip_prefix']):
        #print (i, p['region'], p['service'])
        #print (type(str(i)), str(i))
        c+=1
        if p['service'] in service:
            service[p['service']]+=1
        else:
            service[p['service']]=1
        if str(i) in AWS_ips:
            if p['region'] == AWS_ips[str(i)]:
                pass
            else:
                if p['region']=='GLOBAL':
                    pass
                else:
                    print ('ERROR')
                    print (p)
                    print (str(i), AWS_ips[str(i)])
        else:
            AWS_ips[str(i)] = p['region']

 #       if c > 5000000:
 #           break
    

In [8]:
#print (AWS_ips)
#for i in AWS_ips:
#    print (type(i), i, type(AWS_ips[i]), AWS_ips[i])
#print (json.dumps(AWS_ips, indent=1))
c=0
for i in sorted(cloud_ip_addrs):
    #print (i, cloud_ip_addrs[i])
    c+=1

#print (c)

In [9]:
print ( len(cloud_ip_addrs) )

5937


In [10]:
# MS Azure URLS
# https://download.microsoft.com/download/6/4/D/64DB03BF-895B-4173-A8B1-BA4AD5D4DF22/ServiceTags_AzureGovernment_20220110.json Gov Cloud
# https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20220110.json Public Cloud
url = 'https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20220110.json'
resp = requests.get(url=url)
data = resp.json() # Check the JSON Response Content documentation below


In [11]:
# cloud_ip_addrs = {}

c=0
cidr = ''
ipType = ''
vendor = 'Azure-Public'
region = ''
service = ''

print (type(data))
#print (json.dumps(data, indent=1))
for p in data['values']:
    #print (p)
    region = p['properties']['region']
    service = p['name']
    for c in p['properties']['addressPrefixes']:
        cidr = c
        if c.find(':')!=-1: # ipv6
            ipType = 6
        else:
            ipType = 4
        
        if cidr in cloud_ip_addrs:
#        print ('duplicate cidr', cidr, p, '\nMatches:', cloud_ip_addrs[cidr])
            cloud_ip_addrs[cidr]['addtl'].append( region+':'+service )
        else:
            cloud_ip_addrs[cidr] = {
                "ipType" : ipType,
                "vendor" : vendor,
                "region" : region,
                "service" :service,
                'addtl' : [ region+':'+service ]
            }


<class 'dict'>


In [12]:
url = 'https://download.microsoft.com/download/6/4/D/64DB03BF-895B-4173-A8B1-BA4AD5D4DF22/ServiceTags_AzureGovernment_20220110.json'
resp = requests.get(url=url)
data = resp.json() # Check the JSON Response Content documentation below


# cloud_ip_addrs = {}

c=0
cidr = ''
ipType = ''
vendor = 'Azure-Gov'
region = ''
service = ''

print (type(data))
#print (json.dumps(data, indent=1))
for p in data['values']:
    #print (p)
    region = p['properties']['region']
    service = p['name']
    for c in p['properties']['addressPrefixes']:
        cidr = c
        if c.find(':')!=-1: # ipv6
            ipType = 6
        else:
            ipType = 4
        
        if cidr in cloud_ip_addrs:
#        print ('duplicate cidr', cidr, p, '\nMatches:', cloud_ip_addrs[cidr])
            cloud_ip_addrs[cidr]['addtl'].append( region+':'+service )
        else:
            cloud_ip_addrs[cidr] = {
                "ipType" : ipType,
                "vendor" : vendor,
                "region" : region,
                "service" :service,
                'addtl' : [ region+':'+service ]
            }

<class 'dict'>


In [13]:
c = 0
print (len(cloud_ip_addrs))
for i in cloud_ip_addrs:
    c+=1
    if c > 32000:
        #print (i, cloud_ip_addrs[i])
        for i in range(1000): # Helps reduce iopub rate limit
            pass

36131


In [14]:
#Google Cloud URLS:
# https://www.gstatic.com/ipranges/goog.json
url = 'https://www.gstatic.com/ipranges/goog.json'
resp = requests.get(url=url)
data = resp.json() # Check the JSON Response Content documentation below

In [15]:
print (json.dumps(data, indent=1))

{
 "syncToken": "1642097010289",
 "creationTime": "2022-01-13T10:03:30.289277",
 "prefixes": [
  {
   "ipv4Prefix": "8.8.4.0/24"
  },
  {
   "ipv4Prefix": "8.8.8.0/24"
  },
  {
   "ipv4Prefix": "8.34.208.0/20"
  },
  {
   "ipv4Prefix": "8.35.192.0/20"
  },
  {
   "ipv4Prefix": "23.236.48.0/20"
  },
  {
   "ipv4Prefix": "23.251.128.0/19"
  },
  {
   "ipv4Prefix": "34.64.0.0/10"
  },
  {
   "ipv4Prefix": "34.128.0.0/10"
  },
  {
   "ipv4Prefix": "35.184.0.0/13"
  },
  {
   "ipv4Prefix": "35.192.0.0/14"
  },
  {
   "ipv4Prefix": "35.196.0.0/15"
  },
  {
   "ipv4Prefix": "35.198.0.0/16"
  },
  {
   "ipv4Prefix": "35.199.0.0/17"
  },
  {
   "ipv4Prefix": "35.199.128.0/18"
  },
  {
   "ipv4Prefix": "35.200.0.0/13"
  },
  {
   "ipv4Prefix": "35.208.0.0/12"
  },
  {
   "ipv4Prefix": "35.224.0.0/12"
  },
  {
   "ipv4Prefix": "35.240.0.0/13"
  },
  {
   "ipv4Prefix": "64.15.112.0/20"
  },
  {
   "ipv4Prefix": "64.233.160.0/19"
  },
  {
   "ipv4Prefix": "66.102.0.0/20"
  },
  {
   "ipv4Prefix": "

In [16]:
c=0
cidr = ''
ipType = ''
vendor = 'Google'
region = ''
service = ''

print (type(data))
#print (json.dumps(data, indent=1))
for p in data['prefixes']:
    #print (p)
    region = None
    service = None
    if 'ipv4Prefix' in p:
        cidr = p['ipv4Prefix']
        ipType=4
    else:
        if 'ipv6Prefix' in p:
            cidr = p['ipv6Prefix']
            ipType=6
        else:
            print ('unknow ip type:', p)
        
        if cidr in cloud_ip_addrs:
#        print ('duplicate cidr', cidr, p, '\nMatches:', cloud_ip_addrs[cidr])
            cloud_ip_addrs[cidr]['addtl'].append( 'G:'+region+':'+service )
        else:
            cloud_ip_addrs[cidr] = {
                "ipType" : ipType,
                "vendor" : vendor,
                "region" : region,
                "service" :service,
                'addtl' : [ 'G:' + str(region)+':'+str(service) ]
            }

<class 'dict'>


In [21]:
#            cloud_ip_addrs[cidr] = {
#                "ipType" : ipType,
#                "vendor" : vendor,
#                "region" : region,
#                "service" :service,
#                'addtl' : [ 'G:' + str(region)+':'+str(service) ]
#            }
f = open('cloud_ips_addrs.csv', 'w')
for i in cloud_ip_addrs:
    f.write(i + '~' + str(cloud_ip_addrs[i]['ipType']) + '~' + cloud_ip_addrs[i]['vendor'] + '~' + str(cloud_ip_addrs[i]['region']) + '~' + str(cloud_ip_addrs[i]['service']) + '~' + str(cloud_ip_addrs[i]['addtl'])+ '\n')
f.close()